In [209]:
import random
import numpy as np
from collections import Counter

## Algorithm
- We'll implement genetic algorihtm (GA) for the minimization of an n-parameter real-valued function
- At the first step we create 50 individuals with random parameters
- We use binary tournament selection: in this method we select 2 individuals to compete against eachother, the fittest gets to reproduce.
- From the N/2 individuals left over, if this number is odd we take the fittest and it survives, and pair the rest up randomly into parents
- For reproduction we'll use the WAX Cross-over method. This is where two parents produce N offspring (in our case 200). The genetic information for each child is given by combining the information of the parents, using the WAX equation. We select a random number r = [0,1], and then assign information $a_i = r a_i^{(1)} + (1-r) a_i ^{(2)}$, where $a_i^{(j)}$ is the ith genetic information of the jth parent
- Then we use a 6-fold tournament among the children to choose who survives. The fittest children after this tournament replace their parents.

In [329]:
class Allele:
    
    def __init__(self,data):
        self.data = data
    
    def get_data(self):
        return(self.data)
    
    def update_data(self, input_data):
        self.data = input_data

class Individual:
    
    def __init__(self,chromosome):
        self.chromosome = [Allele(i) for i in chromosome]
        
    def get_chromosome(self):
        return(self.chromosome)
    
    def get_genetic_information(self):
        return([i.get_data() for i in self.chromosome])
    
    def loss_score(self,func):
        return func(self.get_genetic_information())
    
        
    
class Population:
    
    def __init__(self,list_of_individuals):
        self.population_list = list_of_individuals
    
    def get_population_list(self):
        return(self.population_list)
    
    def shuffle_population_into_random_pairs(self):
        random.shuffle(self.population_list)
        pairs = []
        for i in range(int(len(self.population_list)/2)):
            pairs.append((self.population_list[i],self.population_list[i+int(len(self.population_list)/2)]))
        return(pairs)
        
    def select_for_reproduction_by_binary_tournament(self):
        pairs = self.shuffle_population_into_random_pairs()
        to_reproduce = []
        for pair in pairs:
            if pair[0].loss_score(loss_function) > pair[1].loss_score(loss_function):
                to_reproduce.append(pair[0])
            else:
                to_reproduce.append(pair[1])
        self.population_list = to_reproduce
    
    def get_list_of_parents(self):
        self.select_for_reproduction_by_binary_tournament()
        parents = self.shuffle_population_into_random_pairs()
        return(parents)
    

    def get_list_of_children(self,parents):
        parental_info = [i.get_genetic_information() for i in parents]
        children_list = []
        for i in range(200):
            r = random.random()
            atttributes = [r*parental_info[0][j] + (1-r)*parental_info[1][j] for j in range(len(parental_info[0]))]
            children_list.append(Individual(atttributes))
            
        return(children_list)
        #First group into pairs
        #Then remove the one with lowest score from population
        
    
    def do_k_fold_tournament_amongst_list_to_find_two_winners(self, pop_list,k):
        #for N iterations of tournament
        #pick 6 at random, find winner
        #add to "winners list"
        # after N=1000 iterations, 2 entries which appear as winners the most get returned
        winners = []
        count=0
        for tournament in range(1000):
            count +=1
            competitors = random.sample(pop_list,k)
            scores = [competitor.loss_score(loss_function) for competitor in competitors]
            max_index = np.argmax(scores)
            winners.append(competitors[max_index])
            
        final_winners_list = [i[0] for i in Counter(winners).most_common(4)]
        return(final_winners_list)
    
    def replace_parents_with_children(self):
        parent_list = self.get_list_of_parents()
        children = []
        for parents in parent_list:
            children.append(self.get_list_of_children(parents))
        survivors = []
        for children_list in children:
            survivors_from_children = self.do_k_fold_tournament_amongst_list_to_find_two_winners(children_list,6)
            for survivor in survivors_from_children:
                survivors.append(survivor)
        self.population_list = survivors
        
    def perform_N_mutations(self,N):
        for j in range(N):
            self.replace_parents_with_children()
        
        
    def perform_mutation(self):
        # add this after
        a = 1
        return(a)
            

    
    
# Change by hand
def loss_function(params):
    return -(params[0]**2+params[1]**2 + 6 * np.sin(2*params[0])+ 6*np.sin(2*params[1]))



    
    

In [330]:
def create_initial_population_of_size_N(N):
    list_of_individuals=[]
    for i in range(N):
        a = random.uniform(-4,4)
        b = random.uniform(-4,4)
        list_of_individuals.append(Individual([a,b]))
        
    return Population(list_of_individuals)

In [331]:
my_population = create_initial_population_of_size_N(50)

In [332]:
my_population.get_population_list()[0].loss_score(loss_function)

-5.162443711469773

In [334]:
my_population.replace_parents_with_children()

In [335]:
my_population.perform_N_mutations(3)

In [338]:
my_population.get_population_list()[0].get_genetic_information()

[-0.7313798728361667, -0.72593804555387]